## Let's take a closer look at color spaces

You may have remembered we talked about images being stored in RGB (Red Green Blue) color Spaces. Let's take a look at that in OpenCV.

### First thing to remember about OpenCV's RGB is that it's BGR (I know, this is annoying)

Let's look at the image shape again. The '3L' 

In [ ]:
import cv2
import numpy as np

image = cv2.imread('./images/pic.jpg')

### Let's look at the individual color levels for the first pixel (0,0)

In [ ]:
# BGR Values for the first 0,0 pixel
cv2.imshow('image',image)
cv2.waitKey()
B, G, R = image[0,0] 
print (B, G, R)
print (image.shape)
cv2.destroyAllWindows()

Let's see what happens when we convert it to grayscale

In [ ]:
import cv2

# Load image
image = cv2.imread('./images/pic.jpg')

# Convert to grayscale
gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Convert to HSV (as you already had)
HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Now you can safely use gray_img
print("Gray image shape:", gray_img.shape)
print("Pixel value at (496, 600):", gray_img[496, 600])


It's now only 2 dimensions. Each pixel coordinate has only one value (previously 3) with a range of 0 to 255

In [ ]:
gray_img[0, 0]

### Another useful color space is HSV (Hue - Saturation - Value)
Infact HSV is very useful in color filtering.

In [ ]:
import cv2

# Read the input image
image = cv2.imread('./images/pic.jpg')

# Show the original image
cv2.imshow('Original image', image)

# Show each BGR channel separately
cv2.imshow('Blue channel', image[:, :, 0])
cv2.imshow('Green channel', image[:, :, 1])
cv2.imshow('Red channel', image[:, :, 2])

cv2.waitKey(0)
cv2.destroyAllWindows()


### Let's now explore lookng at individual channels in an RGB image

In [ ]:
import cv2

# Load image
image = cv2.imread('./images/pic.jpg')

# Split channels
B, G, R = cv2.split(image)

# Show channels
cv2.imshow("Red Channel", R)
cv2.imshow("Green Channel", G)
cv2.imshow("Blue Channel", B)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Merge back to original
merged = cv2.merge([B, G, R])
cv2.imshow("Merged Original", merged)

# Amplify Blue and Green safely
amplified_B = cv2.add(B, 100)
amplified_G = cv2.add(G, 100)
merged_boosted = cv2.merge([amplified_B, amplified_G, R])

cv2.imshow("Merged with Blue+Green Amplified", merged_boosted)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

B, G, R = cv2.split(image)

# Let's create a matrix of zeros 
# with dimensions of the image h x w  
zeros = np.zeros(image.shape[:2], dtype = "uint8")

cv2.imshow("Red", cv2.merge([zeros, zeros, R]))
cv2.imshow("Green", cv2.merge([zeros, G, zeros]))
cv2.imshow("Blue", cv2.merge([B, zeros, zeros]))

cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'image' is not defined

In [ ]:
image.shape[:2]

#### You can view a list of color converisons here, but keep in mind you won't ever use or need many of these

http://docs.opencv.org/trunk/d7/d1b/group__imgproc__misc.html#ga4e0972be5de079fed4e3a10e24ef5ef0

# Challenge

#### Directions: 
1. Load an image.  Create a filter (as a function) for images, similar to what your present in your cameras. Show and compare each images using different filters you created. Be creative and imaginative. Make sure that the filter adjusts/ corrects the original image.
2. Create or load a grayscale image object. Try to revert it back to an assumed three-channeled image. Tip: Create individual layers and then combine. Additional Tip: You are free to use different values for RGB as long as it should return back to the original grayscale image.
3. Convert and save a BnW image from the grayscale image you created. Save the file as BnW - SURNAME, FirstName.

#### Activity
1. Differentiate RGB and HSV. What do these things do? Is it possible to have one but not the other? 
2. Can I convert HSV images to RGB and vice versa?
3. How are filters useful in image manipulation

In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog

# ---------------- FILTER FUNCTIONS ----------------
def apply_sepia(img):
    kernel = np.array([[0.272, 0.534, 0.131],
                       [0.349, 0.686, 0.168],
                       [0.393, 0.769, 0.189]])
    sepia = cv2.transform(img, kernel)
    return np.clip(sepia, 0, 255).astype(np.uint8)

def apply_negative(img):
    return 255 - img

def apply_brightness(img, value=50):
    return cv2.convertScaleAbs(img, alpha=1, beta=value)

def apply_contrast(img, alpha=1.5):
    return cv2.convertScaleAbs(img, alpha=alpha, beta=0)

def apply_blur(img, ksize=15):
    return cv2.GaussianBlur(img, (ksize, ksize), 0)

# ---------------- MAIN PHOTBOOTH ----------------
class PhotoBooth:
    def __init__(self, root):
        self.root = root
        self.root.title("📸 Photobooth")
        self.image = None
        self.filtered = {}

        # Main menu
        tk.Label(root, text="Welcome to Photobooth!", font=("Arial", 14)).pack(pady=10)
        tk.Button(root, text="📷 Take Photo", width=20, command=self.take_photo).pack(pady=5)
        tk.Button(root, text="🖼 Load Image", width=20, command=self.load_image).pack(pady=5)
        tk.Button(root, text="❌ Exit", width=20, command=root.quit).pack(pady=5)

    def take_photo(self):
        cap = cv2.VideoCapture(0)
        while True:
            ret, frame = cap.read()
            cv2.imshow("Press SPACE to Capture, ESC to Exit", frame)
            key = cv2.waitKey(1)
            if key == 27:  # ESC
                break
            elif key == 32:  # SPACE
                self.image = frame.copy()
                break
        cap.release()
        cv2.destroyAllWindows()
        if self.image is not None:
            self.open_editor()

    def load_image(self):
        path = filedialog.askopenfilename(title="Select an image")
        if path:
            self.image = cv2.imread(path)
            self.open_editor()

    def open_editor(self):
        # Editor window
        editor = tk.Toplevel(self.root)
        editor.title("🎨 Edit Image")

        tk.Label(editor, text="Choose a filter to preview:", font=("Arial", 12)).pack(pady=10)

        tk.Button(editor, text="Sepia", width=20, command=lambda: self.show_filter("Sepia", apply_sepia)).pack(pady=5)
        tk.Button(editor, text="Negative", width=20, command=lambda: self.show_filter("Negative", apply_negative)).pack(pady=5)
        tk.Button(editor, text="Brightness", width=20, command=lambda: self.show_filter("Brightness", lambda img: apply_brightness(img, 60))).pack(pady=5)
        tk.Button(editor, text="Contrast", width=20, command=lambda: self.show_filter("Contrast", lambda img: apply_contrast(img, 1.8))).pack(pady=5)
        tk.Button(editor, text="Blur", width=20, command=lambda: self.show_filter("Blur", apply_blur)).pack(pady=5)

        tk.Button(editor, text="💾 Save Collage", width=20, command=self.save_collage).pack(pady=10)

    def show_filter(self, name, func):
        result = func(self.image)
        self.filtered[name] = result
        cv2.imshow(name, result)

    def save_collage(self):
        if self.image is None:
            return

        # Always include all filters
        sepia = apply_sepia(self.image)
        negative = apply_negative(self.image)
        bright = apply_brightness(self.image, 60)
        contrast = apply_contrast(self.image, 1.8)
        blurred = apply_blur(self.image)

        imgs = [self.image, sepia, negative, bright, contrast, blurred]

        # Resize for collage
        h, w = 250, 250
        imgs = [cv2.resize(img, (w, h)) for img in imgs]

        # Make 3x2 grid
        row1 = np.hstack(imgs[0:3])
        row2 = np.hstack(imgs[3:6])
        collage = np.vstack([row1, row2])

        # Show and save
        cv2.imshow("Photobooth Collage", collage)
        cv2.imwrite("Photobooth_Collage.png", collage)
        print("✅ Collage saved as Photobooth_Collage.png")

        cv2.waitKey(0)
        cv2.destroyAllWindows()

# ---------------- RUN PROGRAM ----------------
if __name__ == "__main__":
    root = tk.Tk()
    app = PhotoBooth(root)
    root.mainloop()
